In [2]:
import cv2, PIL, os, sys
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from cv2 import aruco
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as mpl
import pandas as pd
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%matplotlib nbagg
%load_ext autoreload
%autoreload 2

In [3]:
def load_charuco_images(video_folder:str, every_x_frames:int):
    char_frames = []
    for i, file in enumerate(os.listdir(video_folder)):
        current_frame = -1
        charuco_vid_file_1 = f"{video_folder}/{file}"
        video = cv2.VideoCapture(charuco_vid_file_1)
        grabbed, frame = video.read()
        while grabbed:
            current_frame += 1
            if not current_frame % every_x_frames:
                char_frames.append(frame)
            grabbed, frame = video.read()
    print("number of pics loaded: ", len(char_frames))
    return char_frames  

def save_images_to_folder(folder, frames):   # must manually empty out first to prevent residuals
    if not len(os.listdir(folder)) == len(frames):
        for i, img in enumerate(frames):
            cv2.imwrite(f"{folder}/pic_{i+1}.png", img)

In [7]:
video_folder = ("../../Charuco_06-06-2024")
every_x_frames_chessboard = 10
chessboard_validation_pics = load_charuco_images(f"{video_folder}/intrinsic_vid", every_x_frames_chessboard)

save_images_to_folder(f"{video_folder}/intrinsic_pic", chessboard_validation_pics)

number of vids loaded:  82


In [47]:

chosen_chessboard_image = chessboard_validation_pics[21]

In [8]:

video_folder = "../../Charuco_06-06-2024/intrinsic_vid"
every_x_frames = 10
charucoFrames = []
for filename in os.listdir(video_folder):
    video_path = os.path.join(video_folder, filename)
    video = cv2.VideoCapture(video_path)
    
    if not video.isOpened():
        print(f"Error opening video file {filename}")
        continue

    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT) / every_x_frames)
    current_frame = 0

    for frame_idx in range(0, int(video.get(cv2.CAP_PROP_FRAME_COUNT)), every_x_frames):
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        grabbed, frame = video.read()
        if not grabbed:
            break
        charucoFrames.append(frame)
        current_frame += 1

    video.release()
    print(f"Processed {current_frame} frames in {filename}")

# Ensure charucoFrames is populated with frames
if not charucoFrames:
    raise ValueError("No frames were added to charucoFrames")

print(f"Total frames extracted: {len(charucoFrames)}")


Processed 82 frames in vid_1.mp4
Total frames extracted: 82


In [10]:
for frame in charucoFrames:
    cv2.imshow("Frame", frame)
    cv2.waitKey(10)

cv2.destroyAllWindows()

In [11]:
copiedframes = copy.deepcopy(charucoFrames)
# Ensure the chessboard size is correct (number of inner corners, not squares)
chessboardSize = (5, 8)  # Adjusted to your actual chessboard size
# chessboardSize = (4, 7)  # Adjusted to your actual chessboard size

# Termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(4,7,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1, 2)

size_of_chessboard_squares_mm = 30
objp = objp * size_of_chessboard_squares_mm

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane

# Use the first frame to determine the frame size
if len(copiedframes) > 0:
    frameHeight, frameWidth = copiedframes[0].shape[:2]
    frameSize = (frameWidth, frameHeight)
else:
    raise ValueError("No frames in charucoFrames list")

print(f"Frame size set to: {frameSize}")
found = 0
for idx, frame in enumerate(copiedframes):
    if frame is None:
        print(f"Failed to load frame at index {idx}")
        continue
    
    # Downscale the image by a factor of 5
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, chessboardSize, None)
    # ret, corners = cv2.findChessboardCorners(gray, chessboardSize, flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    if ret:
        found += 1
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(frame, chessboardSize, corners2, ret)
        cv2.imshow('img', frame)
        cv2.waitKey(10)  # Shortened the display time for debugging
    else:
        print(f"Chessboard not found in frame at index {idx}")

cv2.destroyAllWindows()

if len(objpoints) == 0 or len(imgpoints) == 0:
    raise ValueError("No chessboard corners found in any of the frames.")
print(f"Found chessboard corners in {found} frames.")



Frame size set to: (720, 1280)
Chessboard not found in frame at index 0
Chessboard not found in frame at index 1
Chessboard not found in frame at index 2
Chessboard not found in frame at index 3
Chessboard not found in frame at index 4
Chessboard not found in frame at index 5
Chessboard not found in frame at index 6
Chessboard not found in frame at index 7
Chessboard not found in frame at index 8
Chessboard not found in frame at index 9
Chessboard not found in frame at index 10
Chessboard not found in frame at index 11
Chessboard not found in frame at index 12
Chessboard not found in frame at index 13
Chessboard not found in frame at index 14
Chessboard not found in frame at index 15
Chessboard not found in frame at index 16
Chessboard not found in frame at index 17
Chessboard not found in frame at index 18
Chessboard not found in frame at index 19
Chessboard not found in frame at index 20
Chessboard not found in frame at index 21
Chessboard not found in frame at index 22
Chessboard no

ValueError: No chessboard corners found in any of the frames.

In [21]:
for frame in copiedframes:
    cv2.imshow("Frame", frame)
    cv2.waitKey(5)

cv2.destroyAllWindows()

In [55]:
print("Starting camera calibration...")
############## CALIBRATION #######################################################

ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

if not ret:
    raise ValueError("Camera calibration failed.")

# Print calibration results
print("Camera matrix:", cameraMatrix)
print("Distortion coefficients:", dist)
print("Rotation vectors:", rvecs)
print("Translation vectors:", tvecs)

Starting camera calibration...
Camera matrix: [[949.59659461   0.         642.13407395]
 [  0.         953.2563929  361.67848268]
 [  0.           0.           1.        ]]
Distortion coefficients: [[ 2.78050204e-01 -8.92857857e-01  3.45552271e-03 -8.47477709e-04
   9.72786252e-01]]
Rotation vectors: (array([[1.31091816],
       [0.65880591],
       [0.91175629]]), array([[-0.25648364],
       [-1.07208869],
       [-0.0645181 ]]), array([[-0.257118  ],
       [-1.07593723],
       [-0.0560782 ]]), array([[-0.26156344],
       [-1.08119507],
       [-0.0531611 ]]), array([[-0.26304087],
       [-1.07689783],
       [-0.0512144 ]]), array([[-0.26727026],
       [-1.07864915],
       [-0.05037574]]), array([[-0.51148256],
       [-0.14950646],
       [-0.10371859]]), array([[-0.52374099],
       [-0.14971391],
       [-0.11180503]]), array([[-0.52534132],
       [-0.14645009],
       [-0.1163631 ]]), array([[-0.52035122],
       [-0.13693367],
       [-0.10902235]]), array([[-0.51476089]

In [35]:
img = charucoFrames[0]

# Get the current working directory
script_dir = %pwd

# Save the original image
original_img_path = os.path.join(script_dir, 'input.png')
cv2.imwrite(original_img_path, img)

# Perform camera calibration and obtain new camera matrix
h, w = img.shape[:2]
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w, h), 1, (w, h))

# Undistort the image
dst_undistorted = cv2.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# Get the new dimensions after undistortion and cropping
x, y, w, h = roi
new_w = w - x
new_h = h - y

# Crop the undistorted image
dst_cropped = dst_undistorted[y:y+h, x:x+w]

# Save the undistorted and cropped image
undistorted_cropped_img_path = os.path.join(script_dir, 'undistorted_and_cropped.png')
cv2.imwrite(undistorted_cropped_img_path, dst_cropped)

# Perform undistortion with remapping
mapx, mapy = cv2.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w, h), cv2.CV_32FC1)
dst_remap = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# Crop the remapped image
dst_remap_cropped = dst_remap[y:y+h, x:x+w]

# Save the remapped and cropped image
remapped_cropped_img_path = os.path.join(script_dir, 'remapped_and_cropped.png')
cv2.imwrite(remapped_cropped_img_path, dst_remap_cropped)

NameError: name 'cameraMatrix' is not defined

In [17]:
# LaserWorldspacePoints = {"wall1": [0, 0, 57.7], 
#                          "wall2": [0, 67, 57.7], 
#                          "wall3": [0, 75, 57.7], 
#                          "wall4": [0, 142, 57.7], 
#                          "wall5": [0, 150, 57.7], 
#                          "anenometer": [112.5, 62.75, 57.7], 
#                          "cilinder": [110.2, 89.7, 57.7]}
# 
# #not warped
# LaserImagespacePoints = {"wall1": [107, 49], 
#                          "wall2": [580, 64], 
#                          "wall3": [631, 65], 
#                          "wall4": [1029, 78], 
#                          "wall5": [1078, 80], 
#                          "anenometer": [717, 396], 
#                          "cilinder": [1172, 367]}
# 
# #warped image
# LaserImagespacePoints = {"wall1": [107, 49], 
#                          "wall2": [580, 64], 
#                          "wall3": [631, 65], 
#                          "wall4": [1029, 78], 
#                          "wall5": [1078, 80], 
#                          "anenometer": [717, 396], 
#                          "cilinder": [1172, 367]}
# worldspace = np.array([value for value in LaserWorldspacePoints.values()], dtype=np.float32)
# imagespace = np.array([value for value in LaserImagespacePoints.values()], dtype=np.float32)
# 
# worldspace = np.array([(147, -71, 0), (147, 71, 0), (147, 0, 0), (147-110.2, -10, 0)], dtype=np.float32)
# imagespace = np.array([[208, 43][::-1], [1182, 69][::-1], [677, 57][::-1], [436, 337][::-1]], dtype=np.float32)

worldspace = np.float32([(15.8, 10.8, 0), (29.2, 3.9, 0), (36.4, 47.8, 0), (5.2, 59.6, 0)])
imagespace = np.float32([[923, 1487], [142, 908], [1047, 564], [13, 393]])
print(worldspace)
print(imagespace)


[[15.8 10.8  0. ]
 [29.2  3.9  0. ]
 [36.4 47.8  0. ]
 [ 5.2 59.6  0. ]]
[[1487.  923.]
 [ 908.  142.]
 [ 564. 1047.]
 [ 393.   13.]]


In [34]:
# Open video
# cap = cv2.VideoCapture("videos/smoke/needswarping.mp4")
worldspace = np.float32([(15.8, 10.8, 0), (29.2, 3.9, 0), (36.4, 47.8, 0), (5.2, 59.6, 0)])
imagespace = np.float32([[923, 1487], [142, 908], [1047, 564], [13, 393]])
vid_file = "../../Charuco_06-06-2024/intrinsic_vid"
cap = cv2.VideoCapture(f"{vid_file}/{os.listdir(vid_file)[0]}")

# Get frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Extract x, y coordinates and assume the plane normal points towards Z-axis
world_xy = worldspace[:, :2]
plane_normal = np.array([0, 0, 1])

# Calculate the constant term (d) in the plane equation (Ax + By + Cz + d = 0)
d = -worldspace[0][2]  # Use any point on the plane

# Create the plane equation representation
plane_equation = np.concatenate((plane_normal, [d]))

# Define a new virtual camera position
camera_distance = 1000  # Set a reasonable distance
camera_position = np.array([0, 0, -camera_distance])

# Function to project a point onto the plane
def project_onto_plane(point, plane_equation):
    return point - (np.dot(point - camera_position, plane_equation[:3]) / np.linalg.norm(plane_equation[:3])**2) * plane_equation[:3]

# Project world points onto the plane
projected_worldspace = np.array([project_onto_plane(point, plane_equation) for point in worldspace])

# Find minimum and maximum coordinates of projected points
min_x, min_y = projected_worldspace[:, 0].min(), projected_worldspace[:, 1].min()
max_x, max_y = projected_worldspace[:, 0].max(), projected_worldspace[:, 1].max()

# Scale and translate to fit the frame size
scaling_factor = min(frame_width / (max_x - min_x), frame_height / (max_y - min_y))
translation = np.array([-min_x * scaling_factor, -min_y * scaling_factor])

# Function to transform the points
def transform_point(point):
    return (point[:2] * scaling_factor) + translation

# Transform the points
transformed_points = np.array([transform_point(point) for point in projected_worldspace])

# Calculate homography matrix from the original image points to the transformed points
homography_matrix, _ = cv2.findHomography(imagespace, transformed_points)

# Main loop to process the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply the homography transformation to the frame
    warped_frame = cv2.warpPerspective(frame, homography_matrix, (frame_width, frame_height))

    # Display the original and warped frames
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Warped Frame', cv2.resize(warped_frame, (warped_frame.shape[1] // 2, warped_frame.shape[0] // 2)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [67]:
vid_file = "../../Charuco-30-05-2024/video"
ur = cv2.VideoCapture(f"{vid_file}/{os.listdir(vid_file)[0]}")
ur.set(cv2.CAP_PROP_POS_FRAMES, 500)
gr, fr = ur.read()
cv2.imshow("fr", fr)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite(f"../../Charuco-30-05-2024/use_in_paint.png", fr)
cap.release()

True